In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFPSP%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, transient=True)
solverS.output_file = 'solid_outputs/resFPSP%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_temperature_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempS = InterfaceData(size=inodes, value=T_bot)
TempF = InterfaceData(size=inodes, value=300.0)
TempS = InterfaceData(size=inodes, value=T_bot)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=1e-6)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
hF2S = hF[f2s]
hS = solverS.get_interface_h(tag=interface_tag)
hS2F = hS[s2f]

CF1 = hS2F/(hS2F+hF)
CF2 = hF/(hS2F+hF)

CS1 = hS/(hF2S+hS)
CS2 = hF2S/(hF2S+hS)

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FPSP%i.log' % k, mode='w')
flog.write('Fluid pseudo Robin with solid pseudo Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FPSP%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        TempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # retrieve solid ambient temperature
        abTempS.curr[:] = solverS.get_interface_ambient_temperature(
            tag=interface_tag,
            h_given = hS
        )
        
        # update solid interface temperature
        TempS.curr[:] = CS1 * abTempS.curr + CS2 * abTempF.curr[f2s]
        # update solid temperature interface
        solverS.update_temperature_interface(
            tag=interface_tag,
            T=TempS.curr
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid ambient temperature, the new one
        abTempS.curr[:] = solverS.get_interface_ambient_temperature(
            tag=interface_tag,
            h_given = hS
        )
        
        # update fluid interface temperature
        TempF.curr[:] = CF1 * abTempS.curr[s2f] + CF2 * abTempF.curr
        # update residual
        TempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(TempF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(TempF)
            under_relax.update_solution(TempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=TempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=10.
step=1, pc_iterations=6.
step=2, pc_iterations=6.
step=3, pc_iterations=6.
step=4, pc_iterations=6.
step=5, pc_iterations=6.
step=6, pc_iterations=6.
step=7, pc_iterations=6.
step=8, pc_iterations=6.
step=9, pc_iterations=6.
step=10, pc_iterations=5.
step=11, pc_iterations=5.
step=12, pc_iterations=5.
step=13, pc_iterations=5.
step=14, pc_iterations=5.
step=15, pc_iterations=5.
step=16, pc_iterations=5.
step=17, pc_iterations=5.
step=18, pc_iterations=4.
step=19, pc_iterations=4.
step=20, pc_iterations=4.
step=21, pc_iterations=4.
step=22, pc_iterations=3.
step=23, pc_iterations=3.
step=24, pc_iterations=3.
step=25, pc_iterations=3.
step=26, pc_iterations=4.
step=27, pc_iterations=4.
step=28, pc_iterations=4.
step=29, pc_iterations=4.
step=30, pc_iterations=4.
step=31, pc_iterations=4.
step=32, pc_iterations=4.
step=33, pc_iterations=4.
step=34, pc_iterations=4.
step=35, pc_iterations=4.
step=36, pc_iterations=4.
step=37, pc_iterations=4.
step=38, pc_iteration

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFPSP%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.89524364 0.90135687 0.90676573 0.9098468  0.91247003 0.91486497
 0.91696001 0.918833   0.92062502 0.92219227 0.92363861 0.92496297
 0.92622981 0.92738829 0.9284736  0.92947111 0.93042398 0.93130956
 0.93212807 0.93290798 0.93362957 0.93430896 0.93494762 0.93553766
 0.93609775 0.93661622 0.93711245 0.93756428 0.93798158 0.93837742
 0.93874939 0.93909153 0.93941336 0.9397103  0.93998876 0.94024289
 0.94048198 0.94069985 0.94090813 0.94109538 0.94126671 0.94142124
 0.94156371 0.94169453 0.9418131  0.94191925 0.94201517 0.94209907
 0.94217057 0.94223549 0.94228846 0.94233323 0.94236752 0.94239301
 0.94240936 0.94241852 0.9424173  0.94240717 0.94238904 0.94236123
 0.94232236 0.9422776  0.94222135 0.94215707 0.94208041 0.94199538
 0.94189758 0.94178999 0.94166799 0.94153497 0.94139189 0.9412322
 0.94105232 0.94086337 0.94065395 0.94042971 0.94018148 0.93991505
 0.93962306 0.93930977 0.93896587 0.93859382 0.93819255 0.93775944
 0.93728528 0.93676047 0.93620792 0.93559074 0.93490596 0.93416

In [10]:
hF

array([1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219492,
       1958.58219492, 1958.58219492, 1958.58219492, 1958.58219

In [11]:
hS

array([9899.99999996, 9899.99999996, 9899.99999996, 9899.99999996,
       9899.99999996, 9899.99999996, 9899.99999996, 9899.99999996,
       9899.99999996, 9899.99999996, 9899.99999996, 9899.99999996,
       9899.99999996, 9899.99999999, 9900.00000005, 9900.00000007,
       9900.00000007, 9900.00000007, 9900.00000007, 9900.00000007,
       9900.00000007, 9900.00000007, 9900.00000007, 9900.00000007,
       9900.00000007, 9900.00000007, 9900.00000007, 9900.00000007,
       9900.00000007, 9900.00000007, 9900.00000007, 9900.00000007,
       9900.00000007, 9900.00000007, 9900.00000007, 9900.00000007,
       9900.00000007, 9900.00000007, 9900.00000007, 9900.00000007,
       9900.00000007, 9900.00000007, 9900.00000007, 9900.00000007,
       9900.00000007, 9900.00000007, 9900.00000007, 9900.00000007,
       9900.00000007, 9900.00000007, 9900.00000004, 9899.99999998,
       9899.99999996, 9899.99999996, 9899.99999996, 9899.99999996,
       9899.99999996, 9899.99999996, 9899.99999996, 9899.99999